### __Option Pricing on S&P 500 Daily Risk Control 10% Index__

__The S&P 500 Daily Risk Control 10% Index (SPXT10UE) is part of S&P Dow Jones Risk Control Indices familly and use the below parameters__

| ***Index Name*** | ***Underlying Risk Index*** | ***Risk Control Level*** | ***Maximum Leverage*** | ***Interest Rate*** | ***Volatility Calculation*** | ***Return Frequency for Volatility*** | ***Lag to Rebalancing Date*** | ***Decay Factor (Short-Term)*** | ***Decay Factor (Long-Term)*** | ***Rebalancing Frequency*** | ***Launch Date*** | ***Bloomberg Tickers***                                                           |
|------------------------------------------------|------------------------------------------------------|------------------------|----------------------|-------------------------|----------------------------|--------------------------------------|-----------------------------|-------------------------------|------------------------------|---------------------------|---------------|---------------------------------------------------------------------------------|
| S&P 500 Daily Risk Control 10% Index|S&P 500 Total Return: SPTR (USD) | 10% | 150%                 | SOFR + 0.02963*         | Exponentially weighted    | Daily                                | 2 days                      | 94%                           | 97%                          | Daily                     | 10-Sep-09     | ***Excess Return:*** SPXT10UE (USD)|


__The S&P 500 Daily Risk Control indices are computed using the below methodology__

$$
\text{Risk Control ER Index Value}_t 
= 
\text{RiskControlERIndexValue}_{rb}
\,\times\,
\Biggl[
1
\;+\;
K_{rb}\,\biggl(\frac{\text{UnderlyingIndex}_t}{\text{UnderlyingIndex}_{t-1}} \;-\; 1\biggr)
\;-\;
K_{rb}\,\Bigl(
  \prod_{i=rb+1}^{t}\bigl(1 + \text{InterestRate}_{i-1} \times \frac{D_{i-1,i}}{360}\bigr)
  \;-\; 1
\Bigr)
\Biggr]
$$


$$
K_{rb} 
= 
\min\!\Bigl(\text{Max }K,\;\frac{\text{Target Volatility}}{\text{Realized Volatility}_{rb-d}}\Bigr)
$$


$$
\text{RealizedVolatility}_t 
  = \max\bigl(\text{RealizedVolatility}_{S,t}, \text{RealizedVolatility}_{L,t}\bigr)
$$

$$
\text{RealizedVolatility}_{S,t} 
  = \sqrt{\frac{252}{n}\,\text{Variance}_{S,t}}
$$

$$
\text{RealizedVolatility}_{L,t}
  = \sqrt{\frac{252}{n}\,\text{Variance}_{L,t}}
$$

$$
\text{Variance}_{S,t}
  = \lambda_S\,\text{Variance}_{S,t-1}
   + \bigl(1 - \lambda_S\bigr)\,\left[
       \ln\!\Bigl(\frac{\text{UnderlyingIndex}_t}{\text{UnderlyingIndex}_{t-n}}\Bigr)
     \right]^2
$$

$$
\text{Variance}_{L,t}
  = \lambda_L\,\text{Variance}_{L,t-1}
   + \bigl(1 - \lambda_L\bigr)\,\left[
       \ln\!\Bigl(\frac{\text{UnderlyingIndex}_t}{\text{UnderlyingIndex}_{t-n}}\Bigr)
     \right]^2
$$

__Excess Return Volatility Target SDE with SABR Model__


\begin{aligned}
\frac{dV_t}{V_t} &= K_{rb}\Bigl(g \cdot dt + \frac{dF_t}{F_t}\Bigr)
\end{aligned}

\begin{aligned}
\frac{dF_t}{F_t}&= \alpha_t dW_t
\end{aligned}

\begin{aligned}
d\alpha_t &= \nu\, \alpha_t\, dZ_t
\end{aligned}

\begin{aligned}
\mathrm{corr}(dW_t,dZ_t) &= \rho
\end{aligned}

\begin{aligned}
\frac{dS_t}{S_t}&= (r+g)\cdot dt + \frac{dF_t}{F_t}
\end{aligned}

In [2]:
# -------
# IMPORT
# -------
import numpy as np
import pandas as pd
import plotly.express as px
from python_module.pricing_model import BlackScholesModel, SABRModel

In [18]:
# -------
# INPUTS
# -------
F = 100
T = 1
alpha = 0.2 
beta = 1 
rho = -0.9
nu = 0.9
n_steps = 250
n_paths = 1
seed = True
r = 0.03
g = 0.0020
dt = T / n_steps
vol_target = 0.10
max_leverage = 1.5

In [ ]:
eq_process, iv_process = SABRModel.compute_montecarlo(F=F, T=T, alpha=alpha, beta=beta, rho=rho, nu=nu, n_steps=n_steps, n_paths=n_paths, seed=seed)
risky_asset = (eq_process.pct_change() + r*dt + g*dt).fillna(0).add(1).cumprod() * F